In [ ]:
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_dir = '/content/drive/MyDrive/Dataset/Klasifikasi Gambar/train'
test_dir = '/content/drive/MyDrive/Dataset/Klasifikasi Gambar/test'

In [ ]:
import tensorflow as tf

train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=32
)


Found 2876 files belonging to 10 classes.
Found 1193 files belonging to 10 classes.


In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')  # 10 karena Anda punya 10 kelas makanan
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Memuat model MobileNetV2 dengan input shape dan pretrained weights
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,  # Tidak termasuk fully connected layer top
                                               weights='imagenet')

# Bekukan layer pretrained
base_model.trainable = False

# Membuat model sequential dengan base model dan tambahan layers
model = models.Sequential([
    base_model,  # Base model MobileNetV2
    layers.GlobalAveragePooling2D(),  # Pooling untuk mereduksi dimensi
    layers.Dense(128, activation='relu'),  # Fully connected layer dengan 128 unit
    layers.Dense(10, activation='softmax')  # Output layer dengan 10 kelas (sesuaikan dengan jumlah kelas yang diinginkan)
])

# Menyusun model dengan optimizer, loss, dan metric
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.18.0


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Inisialisasi EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=10,  # Tentukan epoch maksimal 10
    callbacks=[early_stopping]
)

Epoch 1/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 434s 5s/step - accuracy: 0.4815 - loss: 1.5355 - val_accuracy: 0.7234 - val_loss: 0.7953
Epoch 2/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 208s 2s/step - accuracy: 0.7922 - loss: 0.6344 - val_accuracy: 0.7728 - val_loss: 0.6811
Epoch 3/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 277s 2s/step - accuracy: 0.8483 - loss: 0.4658 - val_accuracy: 0.7997 - val_loss: 0.5835
Epoch 4/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 253s 2s/step - accuracy: 0.9144 - loss: 0.3260 - val_accuracy: 0.7879 - val_loss: 0.6107
Epoch 5/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 262s 2s/step - accuracy: 0.9342 - loss: 0.2435 - val_accuracy: 0.7829 - val_loss: 0.6315
Epoch 6/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 188s 2s/step - accuracy: 0.9603 - loss: 0.1826 - val_accuracy: 0.7997 - val_loss: 0.6063


In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc}, Test loss: {test_loss}")

38/38 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - accuracy: 0.7977 - loss: 0.5900
Test accuracy: 0.7996647357940674, Test loss: 0.5834572315216064
